# Path Finder

## Setup Spark environment

In [1]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']
get_ipython().run_cell_magic('configure', line="-f", cell='{ "name":"%s-final-path-finder", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5316,application_1618324153128_4558,pyspark,idle,Link,Link,None,
5336,application_1618324153128_4586,pyspark,idle,Link,Link,None,
5342,application_1618324153128_4592,pyspark,idle,Link,Link,None,
5349,application_1618324153128_4602,pyspark,idle,Link,Link,None,
5368,application_1618324153128_4626,pyspark,busy,Link,Link,None,
5376,application_1618324153128_4636,pyspark,idle,Link,Link,None,
5377,application_1618324153128_4637,pyspark,idle,Link,Link,None,
5391,application_1618324153128_4663,pyspark,idle,Link,Link,None,
5392,application_1618324153128_4664,pyspark,idle,Link,Link,None,
5394,application_1618324153128_4666,pyspark,idle,Link,Link,None,


In [2]:
%%send_to_spark -i username -t str -n username

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5419,application_1618324153128_4695,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Successfully passed 'username' as 'username' to Spark kernel

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Import data from HDFS

In [4]:
routes = spark.read.csv('/data/sbb/csv/timetable/routes/2019/05/07/routes.csv', header=True )
stops = spark.read.orc('/data/sbb/orc/geostops')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
walking_edges = spark.read.parquet('/user/%s/final/parquet/walking_edges' % username)
transport_edges = spark.read.parquet('/user/%s/final/parquet/transport_edges' % username)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
transport_edges.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id', 'weekday']

## Shortest Path

In [7]:
inward_walking_edges = walking_edges.groupby('end_vertex').agg({'start_vertex': 'collect_set'})\
                                                              .withColumnRenamed('collect_set(start_vertex)', 'start_vertices')\
                                                              .toPandas().set_index('end_vertex').to_dict()['start_vertices']
walking_edge_duration = walking_edges.toPandas()
walking_edge_duration['key'] = walking_edge_duration.apply(lambda x: (x['start_vertex'], x['end_vertex']), axis=1)
walking_edge_duration = walking_edge_duration.set_index('key').to_dict()['duration']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
edges = transport_edges.filter('weekday == "monday"').sort(F.col('start_time').desc())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# This function assumes a (spark) dataframe with columns 'start_vertex', 'end_vertex', 'start_time', 'duration', 'route_id' 
# edges is a dataframe that represents the edges of the graph, 
# edges should be sorted by order of descending starting time
# inward_walking_edges is a map route_id -> list(route_id), representing the nodes from which we can walk to a given node
# walking_edge_duration is a map (route_id, route_id) -> float, representing the duration of the walk
# end_time should be in the format 'HH:MM'
def latest_departure_paths(edges, end_stop, end_time, inward_walking_edges, walking_edge_duration):
    
    # This function computes whether edge (u, v, t, dur, route_id) can be taken
    # returns update to node u if possible, and None otherwise
    def edge_valid(u, v, time, dur, route_id):
        
        cur_best = next_transfer.get(u)
        next_edge = next_transfer.get(v)
        
        if next_edge: # needs to be defined
            time_v, next_v, route_id_v, dur_v, seq_v = next_edge
            
            if route_id == 'walking':
                if  route_id_v == 'walking' and dur_v + dur <= max_walking_time:
                    time = time_v - dur - transfer_time
                    if (not cur_best) or time > cur_best[0]:
                        return (time, next_v, route_id, dur + dur_v, [u] + seq_v)
                elif route_id_v != 'walking':
                    time = time_v - dur - transfer_time
                    if (not cur_best) or time > cur_best[0]:
                        return (time, v, route_id, dur, [u, v])
                      
            
            elif (route_id == route_id_v or v == end_stop) and time + duration <= time_v: # same route or last_stop
                if (not cur_best) or time > cur_best[0]: # can we improve
                    return (time, next_v, route_id, dur + dur_v, [u] + seq_v)
            
            
            elif time + duration + transfer_time <= time_v: # transfer at node_v
                if (not cur_best) or time > cur_best[0]:
                    return (time, v, route_id, dur, [u, v]) 
        
        # In any other case no update can be made
        return None
                
    def time_to_minutes(timestamp):
        return int(timestamp[:2]) * 60 + int(timestamp[3:5])
    
    transfer_time = 2 # at least 2 minutes to transfer
    max_walking_time = 10

    next_transfer = {} # stores for each node u the node v 
                       # where the next transfer happens format (departure_time, v, route_id to v, duration)
    all_roads= {}
    end_time = time_to_minutes(end_time)
    start_time = end_time - 120 # look only at edges departing at most 2 hours before end_time
    
    next_transfer[end_stop] = (end_time, end_stop, None, 0, [end_stop])
    all_roads[end_stop] = [(end_time, end_stop, None, 0)]

    edges = edges.filter(F.col('start_time').between(start_time, end_time)).toPandas().sort_values(by=['start_time'], ascending=False).to_numpy()

    for row in edges:
        
        start_vertex, end_vertex, start, duration, route_id = row[:5]
        
        update = edge_valid(start_vertex, end_vertex, start, duration, route_id)
        if update:
            if(all_roads.get(start_vertex)!= None):    
                all_roads[start_vertex].append(update)
            elif (all_roads.get(start_vertex)== None):
                all_roads[start_vertex]= [update]
                
            next_transfer[start_vertex] = update
                    
            w_start_vertex = start_vertex
            for w_end_vertex in inward_walking_edges.get(w_start_vertex, []):
                duration = walking_edge_duration[(w_start_vertex, w_end_vertex)]
                w_start = start 
                update = edge_valid(w_start_vertex, w_end_vertex, w_start, duration, 'walking')
                if update:
                    if(all_roads.get(w_start_vertex)!= None):
                        all_roads[w_start_vertex].append(update)
                    elif (all_roads.get(start_vertex)== None):
                        all_roads[w_start_vertex]= [update]
                    
                    next_transfer[w_start_vertex] = update
    
    schema = StructType([
        StructField("start_vertex", StringType(), True),
        StructField("departure_time", StringType(), True),
        StructField("end_vertex", StringType(), True),
        StructField("route_id", StringType(), True),
        StructField("duration", IntegerType(), True),
        StructField("stop_seq", ArrayType(StringType()), True)
    ])
    
    next_transfer_df = spark.createDataFrame([(key,) + value for key, value in next_transfer.items()], schema)

    return next_transfer_df, all_roads

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
import time
start = time.time()
next_transfer_df, all_roads = latest_departure_paths(edges, '8591221', '20:00', inward_walking_edges, walking_edge_duration)
end = time.time() 
print(end - start)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2.59393000603

In [11]:
# This cell is for generating best k paths
def minutes_to_timestamp(minutes):
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)

stop_id_to_stop = stops.select('stop_id', 'stop_name')
stop_id_to_stop_dict = stop_id_to_stop.toPandas().set_index('stop_id').to_dict()['stop_name']
route_id_to_route_name = routes.withColumn('route_name', F.concat(F.col('route_desc'), F.lit(' '), F.col('route_short_name')))\
                               .select('route_id', 'route_desc', 'route_short_name', 'route_name')
route_id_to_route_name_dict = route_id_to_route_name.toPandas().set_index('route_id').to_dict()['route_name']
route_id_to_route_name_dict['walking'] = 'walk'

def select_next(time, paths_from_next_stop):
    rev= paths_from_next_stop[::-1]
    
    for i in rev:
        if (i[0]>=time):
            return i
    
    return (None, None, None, None , [])

def generate_best_k_paths(k_paths, start_stop, k):
    l= k_paths[start_stop]
    nbre_paths= k
    if (l== None):
        print("Sorry bro, no paths available")
        return
    if(len(l)<k):
        nbre_paths= len(l)
    
    
    possible= l[::-1][0:nbre_paths]
    roads= []
    for i in possible: 
        time, next_stop, route_id, duration, stops = i
        journey = []
        while next_stop != start_stop:
            journey.append(
            (minutes_to_timestamp(time),
             stop_id_to_stop_dict[start_stop], 
             route_id_to_route_name_dict[route_id],
             duration,
             minutes_to_timestamp(time + duration),
             stop_id_to_stop_dict[next_stop])
            )
        
            start_stop = next_stop
            paths_from_next_stop = k_paths[next_stop]
            m= select_next(time+duration, paths_from_next_stop) 
            if (m[0]!= None):
                time, next_stop, route_id, duration, stops= m
        
        roads.append(journey)
        
    return roads

def pretty_print_journey(journey):
    if journey== None:
        return "No possible second best path"
    journey_str = '\n'.join(["%s : %s\n|\n| %s (%s min)\n|\n%s : %s\n" % segment for segment in journey])
    return journey_str

g= generate_best_k_paths(all_roads,'8591250', 4 )
for i in g:
    print(pretty_print_journey(i))
    print("**************************")        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
need more than 4 values to unpack
Traceback (most recent call last):
  File "<stdin>", line 51, in generate_best_k_paths
ValueError: need more than 4 values to unpack



In [12]:
m = all_roads['8591250']
time, next_stop, route_id, duration, stops= select_next(4, m)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
stops

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[u'8591250', u'8591118']

## Send to local

Send transfer dataframe to local

In [14]:
%%spark -o next_transfer_df -n -1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%local
next_transfer_df.head()

Reconstruct a dict for transfers to keep old code

In [ ]:
%%local
next_transfer = {e[0]: tuple(e[1:]) for e in next_transfer_df.to_numpy()}

Send route id to route name mapping to local

In [ ]:
%%spark -o route_id_to_route_name -n -1
route_id_to_route_name = routes.withColumn('route_name', F.concat(F.col('route_desc'), F.lit(' '), F.col('route_short_name')))\
                               .select('route_id', 'route_desc', 'route_short_name', 'route_name')

In [ ]:
%%local
route_id_to_route_name_dict = route_id_to_route_name.set_index('route_id').to_dict()['route_name']
route_id_to_route_name_dict['walking'] = 'walk'

Send stop id to stop name mapping to local

In [ ]:
%%spark -o stop_id_to_stop -n -1
stop_id_to_stop = stops.select('stop_id', 'stop_name')

In [ ]:
%%local
stop_id_to_stop_dict = stop_id_to_stop.set_index('stop_id').to_dict()['stop_name']

## Reconstruct path and pretty print

In [ ]:
%%local
def minutes_to_timestamp(minutes):
    return str(minutes // 60) + ':' + str(minutes % 60).zfill(2)
    
    
def reconstruct_journey(next_transfer, start_stop):
    time, next_stop, route_id, duration, stop_seq = next_transfer[start_stop]
    journey = []
    while next_stop != start_stop:
        journey.append(
            (minutes_to_timestamp(time),
             stop_id_to_stop_dict[start_stop], 
             route_id_to_route_name_dict[route_id],
             duration,
             'Ride %d stops' % (len(stop_seq)-1),
             minutes_to_timestamp(time + duration),
             stop_id_to_stop_dict[next_stop])
        )
        
        start_stop = next_stop
        time, next_stop, route_id, duration, stop_seq = next_transfer[next_stop]
    return journey


def pretty_print_journey(journey):
    journey_str = '\n'.join(["%s : %s\n|\n| %s (%s min) %s\n|\n%s : %s\n" % segment for segment in journey])
    return journey_str

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8590883'))) # almost same as sbb

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8591353'))) # almost same as sbb

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8590788'))) # not in sbb

In [ ]:
%%local
print(pretty_print_journey(reconstruct_journey(next_transfer, '8591368'))) # Correct